# Lab 10: Fashion MNIST

---
author: Qiran Hu
date: December 6, 2024
embed-resources: true
---

## Introduction

## Methods

In [1]:
# imports
import numpy as np
import pandas as pd
from joblib import parallel_backend
import seaborn as sns

# data generation and loading
from sklearn.datasets import make_blobs
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# plotting
import matplotlib.pyplot as plt
import matplotlib.colors
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.tree import plot_tree
from sklearn.tree import export_text

# basic classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# basic regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

# ensembles for classification
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# ensembles for regression
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

# model tuning and preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV

# model metrics
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    fbeta_score,
    make_scorer,
    precision_score,
    recall_score,
)
from sklearn.model_selection import GridSearchCV, train_test_split
from pprint import pprint
from sklearn.calibration import CalibratedClassifierCV

import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
# standard imports
import matplotlib.pyplot as plt
import numpy as np
import random

# sklearn data
from sklearn.datasets import make_blobs
from sklearn.datasets import make_circles
from sklearn.datasets import make_classification
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# sklearn models
from sklearn.ensemble import RandomForestClassifier

# sklearn metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

# pytorch imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### Data

In [2]:
# load data
# download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
# summary statistics
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
# define batch size
batch_size = 64

# create train data loader
train_dataloader = DataLoader(
    training_data,
    batch_size=batch_size,
)

# create test data loader
test_dataloader = DataLoader(
    test_data,
    batch_size=batch_size,
)

# check data shapes
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# get cpu, gpu or mps device for training
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device!")

Using mps device!


In [16]:
for image, label in train_dataloader:
    horizontal_pixels = image.shape[-1]  
    print(f"Horizontal pixels: {horizontal_pixels}")

Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal pixels: 28
Horizontal

In [18]:
for image, label in train_dataloader:
    vertical_pixels = image.shape[2]  
    print(f"Vertical pixels: {vertical_pixels}")

Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28
Vertical pixels: 28


In [6]:
# visualizations

### Models

In [7]:
# process data for ML
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 10),
        )

    def forward(self, x):
        logits = self.conv_stack(x)
        return logits
    
model_cnn = ConvNet().to(device)
print(model_cnn)

# define loss function
loss_fn = nn.CrossEntropyLoss()

# define nn optimizer
optimizer_nn = torch.optim.SGD(
    model_cnn.parameters(),
    lr=1e-3,
)

# define cnn optimizer
optimizer_cnn = torch.optim.Adam(
    model_cnn.parameters(),
    lr=0.001,
)

# define train loop
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# define test loop
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

ConvNet(
  (conv_stack): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=3136, out_features=10, bias=True)
  )
)


## Results

In [11]:
# report model metrics
# train model
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_cnn, loss_fn, optimizer_cnn)
    test(test_dataloader, model_cnn, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.290459  [   64/60000]
loss: 0.633721  [ 6464/60000]
loss: 0.281417  [12864/60000]
loss: 0.488642  [19264/60000]
loss: 0.467540  [25664/60000]
loss: 0.466219  [32064/60000]
loss: 0.308177  [38464/60000]
loss: 0.556151  [44864/60000]
loss: 0.454401  [51264/60000]
loss: 0.388495  [57664/60000]
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.370210 

Epoch 2
-------------------------------
loss: 0.228698  [   64/60000]
loss: 0.305704  [ 6464/60000]
loss: 0.229224  [12864/60000]
loss: 0.347507  [19264/60000]
loss: 0.358861  [25664/60000]
loss: 0.388724  [32064/60000]
loss: 0.275800  [38464/60000]
loss: 0.460315  [44864/60000]
loss: 0.334395  [51264/60000]
loss: 0.311511  [57664/60000]
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.340792 

Epoch 3
-------------------------------
loss: 0.212343  [   64/60000]
loss: 0.229814  [ 6464/60000]
loss: 0.178452  [12864/60000]
loss: 0.276353  [19264/60000]
loss: 0.312052  [25664/60000]
loss: 0.348491  [32064/600

In [15]:
# convert the convolutional neural network to TorchScript
model_scripted = torch.jit.script(model_cnn)
# write to disk
model_scripted.save("fashion-mnist.pt")

## Discussion

### Conclusion